In [3]:
import pandas as pd
import numpy as np
import sqlite3
import re
import unicodedata       # Convert char fractions to float
import math              # To deal with NaNs

# Load recipes db and  assign each table to a dataframe

In [68]:
sqlitefile    = 't_project2.sqlite' 

table_name    = 'meal_info'
id_field      = 'meal_id' # name of the ID column
table_name_2  = 'meal_ing'
table_name_3  = 'meal_bow'
column_name   = 'Category'

conn = sqlite3.connect(sqlitefile)
c = conn.cursor()

# Load Meal_info table into meal_df dataframe
c.execute('SELECT * FROM {tn} ORDER BY {idf}'.\
        format(tn=table_name, idf=id_field))
meal_info = pd.DataFrame(c.fetchall())
if len(meal_info.columns) > 8:
    meal_info.drop(0, axis=1, inplace=True)
meal_info.columns = ['Meal_id', 'Meal', 'Rating', 'D', 'Days To Expiration', 'Expertise', 'Spicyness', 'Category']
    
# Load Meal_Ingredients table into meal_ing dataframe
c.execute('SELECT * FROM {tn} ORDER BY {idf}'.\
        format(tn=table_name_2, idf=id_field))
meal_ing = pd.DataFrame(c.fetchall())
if len(meal_ing.columns) > 2:
    meal_ing.drop(0, axis=1, inplace=True)
meal_ing.columns = ['Meal_id','Ingredients']

conn.close()

# Load Bag of Words

In [104]:
fraction_list = ['½', '⅓', '⅔', '¼', '¾', '⅕', '⅖', '⅗', '⅘', '⅙', '⅚', '⅐', '⅛', '⅜', '⅝', '⅞', '⅑', '⅒']
measure_units = ['cups','cup','cloves','clove','fl\.','oz\.','head','sprigs', 'sprig','tsp.','tbsp.']
shape_ing = ['breasts','cubed', 'chops','chopped','chunks','crumbles','florets', 'flakes', 'glaze', 'halves', 'hearts', 
             'kernels','package','paste', 'preserves','pesto','powder','rolls','roll', 
             'seasoning', 'seeds', 'sauce', 'slices','shaved','slivered','with black spots']
type_ing = ['ripe','shredded','bone-in','boneless','skinless']
cat_ing = ['bacon','chorizo','cod','chicken','steak','shrimp','salmon','sausage','beef','turkey','fish','trout','tilapia','pork','veg','prosciutto','meatball','lamb']

In [98]:
meal_info.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 168 entries, 151 to 11
Data columns (total 8 columns):
Meal_id               168 non-null int64
Meal                  168 non-null object
Duration              168 non-null int64
Days To Expiration    168 non-null int64
Expertise             168 non-null object
Spicyness             168 non-null object
Category              168 non-null object
Rating                29 non-null float64
dtypes: float64(1), int64(3), object(4)
memory usage: 16.8+ KB


### Meal_ing

In [99]:
meal_ing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1474 entries, 0 to 1473
Data columns (total 7 columns):
Meal_id        1474 non-null int64
Ingredients    1474 non-null object
Qty            1474 non-null float64
Measure        1092 non-null object
Type           938 non-null object
Ming           1474 non-null object
Shape          236 non-null object
dtypes: float64(1), int64(1), object(5)
memory usage: 80.7+ KB


Do all meals in the meal_ing table refer to their respective meals in table meal_df?

In [72]:
len(meal_ing.Meal_id.unique()) == meal_df.shape[0]

True

# Summarizing Data
## For meal_info: 
#### Duration

In [73]:
# Modify the D column to represent the cooking time but instead of a range, use the average of that range. 
meal_info['Duration'] = meal_info.apply(lambda x: simplify_duration(x['D']),axis=1)
meal_info = meal_info.loc[:,['Meal_id','Meal', 'Rating', 'Duration', 'Days To Expiration', 'Expertise', 'Spicyness', 'Category']].sort_values('Meal')
meal_info.head()

,Meal_id,Meal,Rating,Duration,Days To Expiration,Expertise,Spicyness,Category
151,3907,Argentine Shrimp Chimichurri with Fresno chile...,5,55,3,easy,medium,Shrimp
0,2066,Artichoke and Tomato Flatbread with goat chees...,None,47,7,easy,mild,vegetarian
134,3786,Avocado and Tofu Sushi Bowl with carrots cucum...,None,62,3,intermediate,medium,vegetarian
101,3522,BBQ Shrimp Pizza with pico de gallo,None,47,3,easy,medium,seafood
56,3246,BBQ Turkey Burger with cheddar caramelized oni...,None,70,5,intermediate,not,poultry


#### Rating

In [74]:
# Convert star to string to float. 
meal_info['s1']=[np.nan if val == 'None' else float(val) for val in meal_info.Rating.values]
meal_info = meal_info.drop('Rating', axis=1)
meal_info.rename(columns={'s1': 'Rating'}, inplace=True)
meal_info.head()

,Meal_id,Meal,Duration,Days To Expiration,Expertise,Spicyness,Category,Rating
151,3907,Argentine Shrimp Chimichurri with Fresno chile...,55,3,easy,medium,Shrimp,5.0
0,2066,Artichoke and Tomato Flatbread with goat chees...,47,7,easy,mild,vegetarian,NaN
134,3786,Avocado and Tofu Sushi Bowl with carrots cucum...,62,3,intermediate,medium,vegetarian,NaN
101,3522,BBQ Shrimp Pizza with pico de gallo,47,3,easy,medium,seafood,NaN
56,3246,BBQ Turkey Burger with cheddar caramelized oni...,70,5,intermediate,not,poultry,NaN


## Segment data on the meal_ing table
The ingredients will segment in: 
    1. quantity (a number or fraction)
    2. Unit of measure
    3. Type or Adjective
    4. Main ingredient
    5. Shape or Texture or Part

In [75]:
# The new column: RAW_INGREDIENTS will have the ingredient segments separated by \t
meal_ing['raw_ingredients'] = meal_ing.Ingredients.apply(lambda x: identify_ingredients(x))
meal_ing['Qty']     = meal_ing.raw_ingredients.apply(lambda x: get_part_of_ingredient(0,x))
meal_ing['Measure'] = meal_ing.raw_ingredients.apply(lambda x: get_part_of_ingredient(1,x))
meal_ing['Type']    = meal_ing.raw_ingredients.apply(lambda x: get_part_of_ingredient(2,x))
meal_ing['Ming']    = meal_ing.raw_ingredients.apply(lambda x: get_part_of_ingredient(3,x))
meal_ing['Shape']   = meal_ing.raw_ingredients.apply(lambda x: get_part_of_ingredient(4,x))

meal_ing = meal_ing.iloc[:,[0,1,3,4,5,6,7]]

In [120]:
meal_ing.head()

,Meal_id,Ingredients,Qty,Measure,Type,Ming,Shape
0,2066,2 garlic cloves,2.0,cloves,NaN,garlic,NaN
1,2066,4 oz. grape tomatoes,4.0,oz.,grape,tomatoes,NaN
2,2066,3 oz. artichoke hearts,3.0,oz.,NaN,artichoke,hearts
3,2066,1 shallot,1.0,NaN,NaN,shallot,NaN
4,2066,4 pepperoncini,4.0,NaN,NaN,pepperoncini,NaN


#### Quantity

In [77]:
# Convert Qty column (string) values to int or float
meal_ing['Qty'] = meal_ing.Qty.apply(lambda x: convert_char_fraction_to_float(x))
meal_ing.head()

,Meal_id,Ingredients,Qty,Measure,Type,Ming,Shape
0,2066,2 garlic cloves,2.0,cloves,NaN,garlic,NaN
1,2066,4 oz. grape tomatoes,4.0,oz.,grape,tomatoes,NaN
2,2066,3 oz. artichoke hearts,3.0,oz.,NaN,artichoke,hearts
3,2066,1 shallot,1.0,NaN,NaN,shallot,NaN
4,2066,4 pepperoncini,4.0,NaN,NaN,pepperoncini,NaN


### Get the category of the meal based on it's title

In [113]:
meal_info['Ctgry'] = meal_info.Meal.apply(lambda x: get_category(x))
meal_info.drop('Category', axis=1, inplace=True)
# Veg to Vegetarian. 
meal_info.loc[meal_info.cat == 'veg', ['cat']] = 'vegetarian'
meal_info.loc[meal_info.cat == 'bacon steak', ['cat']] = 'steak'
meal_info.loc[meal_info.cat == 'turkey meatball', ['cat']] = 'turkey'
meal_info.loc[meal_info.cat == 'pork meatball', ['cat']] = 'pork'
meal_info.loc[(meal_info.cat == 'salmon')|(meal_info.cat == 'tilapia')|(meal_info.cat == 'cod') | (meal_info.cat == 'trout'), ['cat']] = 'fish'
meal_info.loc[(meal_info.cat == 'fish'), ['cat']] = 'seafood'
meal_info.loc[meal_info.cat == 'chorizo', ['cat']] = 'prosciutto'
meal_info.loc[(meal_info.cat == 'chicken')|(meal_info.cat == 'turkey'), ['cat']] = 'poultry'

,Meal_id,Meal,Duration,Days To Expiration,Expertise,Spicyness,Category,Rating,cat
151,3907,Argentine Shrimp Chimichurri with Fresno chile...,55,3,easy,medium,Shrimp,5.0,shrimp
0,2066,Artichoke and Tomato Flatbread with goat chees...,47,7,easy,mild,vegetarian,NaN,vegetarian
134,3786,Avocado and Tofu Sushi Bowl with carrots cucum...,62,3,intermediate,medium,vegetarian,NaN,vegetarian
101,3522,BBQ Shrimp Pizza with pico de gallo,47,3,easy,medium,seafood,NaN,shrimp
56,3246,BBQ Turkey Burger with cheddar caramelized oni...,70,5,intermediate,not,poultry,NaN,turkey


### Main Ingredients (a.k.a. Ming)

In [ ]:
# Fusion singular with plural ingredients (limes -> lime)
# Since the occurrence of every ingredient is an relevant measure, 
# it's necessary to consider the ingredient's plural and singular form as equal
meal_ing = fusion_plural_to_singular_ingredients(meal_ing)

# Refining ingredients outliers. 
meal_ing.loc[meal_ing.Ming == 'gluten-free', 'Type'] = gluten-free
meal_ing.loc[meal_ing.Ming == 'gluten-free', 'Ming'] = miso
meal_ing.loc[meal_ing.Ming == 'heart', 'Type'] = None
meal_ing.loc[meal_ing.Ming == 'heart', 'Shape'] = 'heart'
meal_ing.loc[meal_ing.Ming == 'heart', 'Ming'] = 'romaine'
meal_ing.loc[meal_ing.Ming == 'pepper', 'Ming'] = 'peppers'
meal_ing.loc[meal_ing.Ming == 'shredded', 'Type'] = 'cheddar'
meal_ing.loc[meal_ing.Ming == 'shredded', 'Shape'] = 'shredded'
meal_ing.loc[meal_ing.Ming == 'shredded', 'Ming'] = 'cheese'
meal_ing.loc[meal_ing.Ming == 'squash,', 'Ming'] = 'squash'
meal_ing.loc[meal_ing.Ming == 'strawberries', 'Ming'] = 'strawberry'
meal_ing.loc[meal_ing.Ming == 'fillets', 'Shape'] = 'fillets'
meal_ing.loc[meal_ing.Shape == 'fillets', 'Ming'] = meal_ing.loc[meal_ing.Shape == 'fillets', 'Type']
meal_ing.loc[meal_ing.Shape == 'fillets', 'Type'] = None
meal_ing.loc[meal_ing.Ming == 'pieces', 'Shape'] = 'pieces'
meal_ing.loc[meal_ing.Ming == 'pieces', 'Type'] = None
meal_ing.loc[meal_ing.Ming == 'pieces', 'Ming'] = 'pecan'
meal_ing.loc[meal_ing.Ming == 'gouda', 'Type'] = 'smoked gouda'
meal_ing.loc[meal_ing.Ming == 'gouda', 'Ming'] = 'cheese'
meal_ing.loc[meal_ing.Ming == 'mozzarella', 'Shape'] = meal_ing.loc[meal_ing.Ming == 'mozzarella', 'Type']
meal_ing.loc[meal_ing.Ming == 'mozzarella', 'Type'] = 'mozzarella'
meal_ing.loc[meal_ing.Ming == 'mozzarella', 'Ming'] = 'cheese'
meal_ing.loc[meal_ing.Type == 'grated', 'Shape'] = 'grated'
meal_ing.loc[meal_ing.Type == 'grated', 'Ming'] = 'cheese'
meal_ing.loc[meal_ing.Type == 'grated', 'Type'] = 'parmesan'
meal_ing.loc[meal_ing.Ming == 'steel trout', 'Type'] = 'steelhead'
meal_ing.loc[meal_ing.Ming == 'steel trout', 'Shape'] = None
meal_ing.loc[meal_ing.Ming == 'steel trout', 'Ming'] = 'trout'

# Build the ingredients dataframe

In [242]:
ingredients = build_ingredients_dataframe(meal_ing)
ingredients.shape

(188, 2)

# Post Processing: 
### Download the 3 dataframes from the project2.sqlite

In [82]:
meal_info, meal_ing, ingredients = load_3_dataframes()

### Update the Ingredients table

In [165]:
ingredients = build_ingredients_dataframe(meal_ing)

### Save dataframes to sqlite db

In [240]:
save_3_tables('t_project2.sqlite', meal_info, meal_ing, ingredients)

Tables Saved to t_project2.sqlite


/Users/iZbra1/anaconda/lib/python3.6/site-packages/pandas/core/generic.py:1534: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  chunksize=chunksize, dtype=dtype)


# Functions

In [206]:
#########################################################
#       FUNCTIONS DEALING MEAL_INFO (FORMERLY MEAL_DF)
#########################################################
'''Gets the category of the meal by searching in the meal title'''
def get_category(title):
    cat_pattern     = r""+'|'.join(cat_ing)
#    for title in meal_info.Meal:
    r = re.findall(cat_pattern, title.lower())
    if r:
        shp = ' '.join(r)
        return shp
    else: 
        return 'vegetarian'
    '''
simplify_duration
    Sets Duration value to its average instead of a range. 
'''
def simplify_duration(d):
    d = d.split('-')
    if len(d)>1:
        return int(int(d[1])+int(d[0])/2)
    else: 
        return int(d[0][:-1])

#########################################################
#       FUNCTIONS DEALING WITH THE DATABASE
#########################################################
'''
save_3_tables(dbname, meal_info, meal_ing, ingredients):
    Saves the 3 tables in the dbname.sqlite. 
    
load_3_dataframes :
    Receives the name of the sqlitefile
    Returns the specific 3 tables: meal_info, meal_ing and ingredients 
    as dataframes. 

show_db_tables
    Receives as parameter the name of the sqlite file 
    Prints out the tables the database has. 

build_create_query
    Receives as parameters: 
        table_name: Name of the table to be created in sqlite database
        dataframe:  The dataframe destined to be in the table. 
        flag_id:    If true, primary key is set to the index
    
    Returns a the fashioned query capable to create the requested table
'''
def save_3_tables(db_name, meal_info, meal_ing, ingredients):
    conn = sqlite3.connect(db_name)
    meal_info.to_sql("meal_info", conn, if_exists="replace")
    meal_ing.to_sql("meal_ing", conn, if_exists="replace")
    ingredients.to_sql("ingredients", conn, if_exists="replace")
    conn.commit()
    conn.close()
    print('Tables Saved to',db_name)

def show_db_tables(db_name):
    conn = sqlite3.connect(db_name)
    c = conn.cursor()
    query_check  = 'SELECT name FROM sqlite_master WHERE type= "table";'
    c.execute(query_check)
    table_set = [item[0] for item in c.fetchall()]
    print(table_set)

def build_create_query(table_name, df, flag_id):
    my_query = 'CREATE TABLE'+' '+table_name+' '+'('
    for ind, col in enumerate(df.columns):
        col_ = re.sub(' ','_',col) 
        # Get a real sample of the column value to determine its data type
        my_s = df[col][~df[col].isnull()]
        val_ = my_s.values[0]  
        if isinstance(val_, str):
            type_name = ' TEXT'
        elif isinstance(val_, np.int64):
            type_name = ' INTEGER'
        else: 
            type_name = ' REAL'
        if flag_id == True and ind == 0:
            pk = ' PRIMARY KEY,'
        else: 
            pk = ','
        my_query += ' ' + col_.lower() + type_name + pk
    return my_query.rstrip(',')+')'

def load_3_dataframes(table_name_1, table_name_2, table_name_3, db_name = 'project2.sqlite'):
    id_field      = 'meal_id'
    q1 = 'SELECT * FROM {tn} ORDER BY {idf}'.format(tn=table_name_1, idf=id_field)
    q2 = 'SELECT * FROM {tn} ORDER BY {idf}'.format(tn=table_name_2, idf=id_field)
    q3 = 'SELECT * FROM {tn}'.format(tn=table_name_3)
    conn = sqlite3.connect(db_name)
    # Load Meal_info Dataframe
    meal_info = pd.read_sql_query(q1, conn)
    try: 
        meal_info = meal_info.drop('index', axis=1)
    except: 
        pass
    # Load Meal_ing Dataframe
    meal_ing = pd.read_sql_query(q2, conn)
    try:
        meal_ing = meal_ing.drop('index', axis=1)
    except: 
        pass
    # Load Ingredients Dataframe
    t3 = pd.read_sql_query(q3, conn)
    try: 
        t3 = t3.drop('index', axis=1)
    except:
        pass
    conn.close()
    return meal_info, meal_ing, t3

#########################################################
#        FUNCTIONS TO PROCESS THE INGREDIENTS             
#########################################################
'''
identify_ingredients
    Receives the ingredients as a whole sentence and divides it in said smaller segments. 

get_part_of_ingredient
    This function splits the string s1 by tabs 
    (i.e. qty\tmsr\ttyp\ting\tshp) and returns the item on the indx place. 

fusion_plural_to_singular_ingredients 
    Receives as parameter the dataframe meal_ing
    Modifies every main ingredient if it has a singular and plural form by changing 
    the plural form to its singular homologue. 

convert_char_fraction_to_float
    Receives as parameter 

build_ingredients_dataframe(meal_ing)
    Gets as parameter the meal_ingredients dataframe and 
    selects the unique main ingredients 
    to store it in a new dataframe along with a unique Id of 4 characters, 
        1st: first letter of the ingredient
        2nd: length of the ingredient (if length is 12 it will be reduced to 1+2=3)
        3rd: ord(last letter)
        4th: number of row
'''

def identify_ingredients(ing):
# The next variables will define the pattern to search in the ingredients segment 
# for identification 
    quantity_pattern = r'\bfrac\d+|\d\.\d+|\d+|'+'|'.join(fraction_list)
    unit_pattern     = r"'\bear\b'|"+'|'.join(measure_units)
    shape_pattern    = r""+'|'.join(shape_ing)
    type_pattern     = r""+'|'.join(type_ing)
    qty = msr = shp = typ = '' 
    # Get the QTY
    r = re.search(r"&|;", ing)
    if r:
        # Correct the bug of 1&frac45 numbers
        f = ing.split()[0]
        if r.start() == 0:
            qty = round(int(f[-3])/int(f[-2]), 1)
        else:
            qty = round(int(f[0])+(int(f[-3])/int(f[-2])),1)
        ing = ing[len(f):].lstrip(' ')
    else:
        r = re.search(quantity_pattern, ing)
        qty = ing[r.start():r.end()]
        ing = ing[r.end()+1:]       
    # Get rid of anything within parenthesis 
    r = re.search(r"\(.{1,10}\)", ing)
    if r:
        ing = ing[:r.start()-1]
    # GET Unit of Measure        
    r = re.findall(unit_pattern,ing)
    if r:
        msr = ' '.join(r)
        temp_ing = ing.split(msr)
        ing = ''.join(temp_ing).lstrip(' ').rstrip(' ')
    # Get the texture or shape 
    if len(ing.split()) > 1:
        r = re.findall(shape_pattern, ing)
        if r:
            shp = ' '.join(r)
            temp_ing = ing.split(shp)
            ing = ''.join(temp_ing).lstrip(' ').rstrip(' ')
    # Assign the remainings to type and main_ingredient variables. 
    if len(ing.split()) > 1:
        typ = ' '.join(ing.split()[:-1])
        temp_ing = ing.split()[-1]
        ing = ''.join(temp_ing).lstrip(' ').rstrip(' ')
    return str(qty)+'\t'+msr+'\t'+typ+'\t'+ing+'\t'+shp

def get_part_of_ingredient(indx, s1):
    s = s1.split('\t')
    if s[indx] != '':
        return s[indx]
    else: 
        return np.nan

def convert_char_fraction_to_float(frac_str):
    try:
        return float(frac_str) 
    except:
        return unicodedata.numeric(frac_str)
    
def fusion_plural_to_singular_ingredients(meal_ing):
    s = meal_ing.loc[:,['Meal_id','Ming']].sort_values('Ming').reset_index()
    for ind in range(len(s)-1):
        if s.loc[ind,'Ming']+'s'==s.loc[ind+1,'Ming'] or s.loc[ind,'Ming']+'es'==s.loc[ind+1,'Ming']:
            a = s.loc[ind,'Ming']
            b = s.loc[ind+1,'Ming']
            meal_ing.loc[(meal_ing.Ming == a) | (meal_ing.Ming == b),'Ming'] = a
    return meal_ing
    
def build_ingredients_dataframe(meal_ing):
    # Getting a list of unique ingredients. 
    i2 = meal_ing.Ming.sort_values().unique()
    # Cleaning data, getting rid of everything that is not text.
    i3 = re.findall(r"\w+-\w+|\w+", ' '.join(i2))
    # Converting it back to a Series
    ingredients = pd.DataFrame(i3, columns = ['Ming'])
    # Adding a unique ID
    ingredients['ing_id']=[row[:2]+str(len(row))+str(row[-1])+str(ind) for ind, row in enumerate(ingredients['Ming'])]
    return ingredients
#########################################################
#
#########################################################